# Ex3: NLP - Tags 

### Requirement: Build a tags filter. Use the various NLP tools and a classifier, to predict tag for one question.  In future questions could be auto-tagged by such a classifier or tags could be recommended to users prior to posting.
- Dataset: stack-overflow-data.csv. It contains Stack Overflow questions and associated tags.
- Link tham khảo: http://benalexkeen.com/multiclass-text-classification-with-pyspark/

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

In [ ]:
SparkContext.setSystemProperty('spark.executor.memory', '6g')
sc = SparkContext(master='spark://172.25.53.2:7077', appName='Stack_Overflow')

In [ ]:
sc

In [ ]:
spark = SparkSession(sc)

In [ ]:
file_name = "hdfs://172.24.40.251:19000/stack_overflow_data.csv"
# file_name = "stack-overflow-data.csv"

In [ ]:
data = spark.read.csv(file_name, inferSchema=True,header=True)

In [ ]:
data.show(5)

In [ ]:
data.groupby('tags').count().show(30)

In [ ]:
tags_null_data = data.filter(data.tags.isNull())

In [ ]:
tags_null_data.count()

In [ ]:
data = data.filter(data.tags.isNotNull())

In [ ]:
data.count()

In [ ]:
from pyspark.sql.functions import *

## Clean and Prepare the Data

** Create a new length feature: **

In [ ]:
from pyspark.sql.functions import length

In [ ]:
data = data.withColumn('length',length(data['post']))

In [ ]:
data.show()

In [ ]:
# Pretty Clear Difference
data.groupby('tags').mean().show()

## Feature Transformations

In [ ]:
from bs4 import BeautifulSoup

from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [ ]:
class BsTextExtractor(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(BsTextExtractor, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, dataset):

        def f(s):
            cleaned_post = BeautifulSoup(s).text
            return cleaned_post

        t = StringType()
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
text_extractor = BsTextExtractor(inputCol="post", outputCol="cleaned_post")
tokenizer = Tokenizer(inputCol="cleaned_post", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
class_to_num = StringIndexer(inputCol='tags',outputCol='label')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [ ]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
data_prep_pipe = Pipeline(stages=[class_to_num,text_extractor,tokenizer,stopremove,count_vec,idf,clean_up])

In [ ]:
cleaner = data_prep_pipe.fit(data)

In [ ]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [ ]:
clean_data = clean_data.select(['label','features'])

In [ ]:
clean_data.show() 

In [ ]:
(training,testing) = clean_data.randomSplit([0.7,0.3], seed=142)

In [ ]:
#training.cache()

In [ ]:
#testing.cache()

In [ ]:
#training.groupBy("label").count().show()

In [ ]:
#testing.groupBy("label").count().show()

In [ ]:
predictor = nb.fit(training)

In [ ]:
test_results = predictor.transform(testing)

In [ ]:
test_results.show()

In [ ]:
# Create a confusion matrix
test_results.groupBy('label', 'prediction').count().show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting: {}".format(acc))

In [ ]:
# save may cuc bo
nb.save("NB_TagFilters_model")

In [ ]:
# save hdfs
nb.save("hdfs://172.24.40.251:19000/NB_TagFilters_model")

- Not very good result! (~72%)
- Solution: Try switching out the classification models! Or even try to come up with other engineered features!...

### Use LogisticRegression/Random Forest

### Logistic Regression

In [ ]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

In [ ]:
lg = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

In [ ]:
predictor_1 = lg.fit(training)

In [ ]:
test_results_1 = predictor_1.transform(testing)

In [ ]:
# Create a confusion matrix
test_results_1.groupBy('label', 'prediction').count().show()

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc_1 = acc_eval.evaluate(test_results_1)
print("Accuracy of model at predicting: {}".format(acc_1))

In [ ]:
## It's not better result!!!

In [ ]:
# Save máy cục bộ
# lg.save("LG_TagFilters_model")

In [ ]:
# Save HDFS
# lg.save("hdfs://172.24.40.251:19000/LG_TagFilters_model")

### Random forest

In [ ]:
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 500, \
                            maxDepth = 5, \
                            maxBins = 64)

In [ ]:
predictor_2 = rf.fit(training)

In [ ]:
test_results_2 = predictor_2.transform(testing)

In [ ]:
# Create a confusion matrix
test_results_2.groupBy('label', 'prediction').count().show()

In [ ]:
test_results_2.groupBy('prediction').count().show()

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc_2 = acc_eval.evaluate(test_results_2)
print("Accuracy of model at predicting: {}".format(acc_2))

In [ ]:
## It has higher accuracy but is not a better result!!!

In [ ]:
# Save máy cục bộ
# rf.save("RF_TagFilters_model")

In [ ]:
# Save HDFS
# rf.save("hdfs://172.24.40.251:19000/RF_TagFilters_model")

In [ ]:
sc.stop()